In [ ]:
path_to_muograph = "../"
import sys
sys.path.insert(1, path_to_muograph)

![alternative text](../images/muograph_logo.png)


**<h1><center>Hits tutorial</center></h1>**

The `Hits` class is used to **store** and **process muon hits** on detector planes and  to **simulate** basic **detector reasponse**: spatial resolution and efficiency.

**SUMMARY**

1. *Initialization*

    - Accepts **muon hit** and **energy data** either from a `CSV` file or a `Pandas DataFrame`.
    - Supports **unit conversions** for spatial data (e.g., mm, cm, dm, m).
    - Allows **event filtering** based on muon energy.

2. *Detector effects*:

    - Models detector **efficiency** for hit recording and computes a muon detection probability.
    - Models detector **spatial resolution** for hit recording by adding Gaussian noise.

3. *Data*:

    - Stores hits as 3D **tensors** to leverage tensors and **GPU** **acceleration**.
    - Stores **generation level hits** (`gen_hits`) and **reconstructed hits** (after efficiency and spatial resolution simulation `reco_hits`) separatly.

4. *Visualization*:

    - Provides a **plot method** to create 2D histograms of hits on specified detector panels.

The hits are currently red from either a `.csv` file or a `pandas.DataFrame`. Other file format will be supported in the future.


The `Hits` class takes the following arguments:

 - `csv_filename: str` The file path to the CSV containing hit and energy data. Either `csv_filename` or `df` must be provided, but not both.


 - `df (Optional[pd.DataFrame])`: A DataFrame containing hit and energy data. Use this instead of loading data from a CSV file.


#### **IMPORTANT:**

The `.csv` file or `pd.DataFrame` must have spcific column labels, corresponding to the hit coordinate ($x, y, z$) and the label of the plane, e.g `X0` is the muon hit $x$ position on plane $0$. 

Additionally, the muon's energy can be provided.

Below is an example of `.csv` file heading:

In [ ]:
csv_file = '../muograph/data/iron_barrel/barrel_and_cubes_scattering.csv'

import pandas as pd
dataframe = pd.read_csv(csv_file)
dataframe.columns

In [ ]:
dataframe.head()

Other arguments are:

 - `plane_labels (Optional[Tuple[int, ...]])`: Specifies the plane labels to include from the data, as a tuple of integers. Only hits from these planes will be loaded if provided.

 - `spatial_res (Optional[Tuple[float, float, float]])`: The spatial resolution of detector panels along the x, y, and z axes, in units specified by `input_unit`. Assumes uniform resolution across all panels if provided.

 - `energy_range (Optional[Tuple[float, float]])`: A tuple specifying the minimum and maximum energy range for hits to be included. Only hits within this range will be processed if provided.

 - `efficiency (float)`: The efficiency factor of the detector panels, applied uniformly across all panels. Defaults to 1.0, representing full efficiency.

 - `input_unit (str)`: The unit of measurement for the input data (e.g., "mm", "cm"). Data will be rescaled to millimeters if another unit is specified. Defaults to "mm".

## **I - Perfect resolution example**

Let's instanciate the `Hits` class, using hits corresponding to the 3 first plane of the detector (panels placed above the volume of interest).

In this example, **no spatial resolution** nor **efficiency** is simulated.

In [ ]:
from muograph.hits.hits import Hits

hits_above = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (0, 1, 2),  # Include hits from first planes 0, 1 and 2
    input_unit="mm",  # The distance unit in the csv file.
)

In [ ]:
hits_above

The `Hits` class conveniently stores the hits in a Pytorch `Tensor`, with shape `(3, n_panels, n_mu)`:

In [ ]:
hits_above.gen_hits, hits_above.gen_hits.shape

In [ ]:
event = 120
plane_label = 0

(
    f"Muon #{event} hits detector panel {plane_label} "
    f"at x, y, z = {hits_above.gen_hits[0, plane_label, event]:.1f},"
    f" {hits_above.gen_hits[1, plane_label, event]:.1f},"
    f" {hits_above.gen_hits[2, plane_label, event]:.1f} mm "
)


Muon hits can be plotted on a 2D histogram, using the `plot` method:

In [ ]:
hits_above.plot(plane_label = 0)
hits_above.plot(plane_label = 1)
hits_above.plot(plane_label = 2)

Because no spatial resolution value was used as argument, the generation level hits `gen_hits` and reconstructed hits `reco_hits` are identical:

In [ ]:
diff = (hits_above.gen_hits - hits_above.reco_hits).unique()
diff

## **II - 1mm spatial resolution example**

Now let's **simulate** a **1 mm spatial resolution** along the horizontal direction ($x, y$ axis), by providing a `spatial_res` argument.

Assuming horizontal planes, we choose a spatial resolution of 1 mm along the $x$ and $y$ axis. The vertical coordinate $z$ is left unchanged.

In [ ]:
hits_1mm = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (0, 1, 2),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
    spatial_res = (1.0, 1.0, 0.)  # The spatial resolution along x, y and z in mm.
)

The **reconstructed hits** `reco-hits` are computed by adding **Gaussian noise** to the true hits `gen_hits`:

$$
xyz_{\mathrm{reco}} = xyz_{\mathrm{gen}} + \mathcal{G}(\mu=0, \sigma = \sigma_{xyz}) 
$$

with $\mathcal{G}$ a Guassian distribution with mean $\mu = 0$ and standard deviation $\sigma$.

The **spatial resolution** is assumed to be **uniform** across the whole panel's area.

In [ ]:
# Copmute the difference between true and reconstructed hits
diff = (hits_1mm.gen_hits - hits_1mm.reco_hits)
diff_x, diff_y, diff_z = diff[0].ravel(), diff[1].ravel(), diff[2].ravel()

In [ ]:
from muograph.plotting.plotting import plot_hist
plot_hist(diff_x, xlabel=r"error on $x$ [mm]")
plot_hist(diff_y, xlabel=r"error on $y$ [mm]")
plot_hist(diff_z, xlabel=r"error on $z$ [mm]")


The **effect** of **spatial resolution** on the reconstructed **tracks** will be treated in **tutorial 2**: `02_Tracking.ipynb`.

## **III - 90% efficiency example**

Now let's **simulate** a 90% individual panel **efficiency** by providing an efficiency argument.

In [ ]:
hits_90eff = Hits(
    csv_filename = csv_file,  # The csv file
    plane_labels = (0, 1, 2),  # Include hits from first planes 0, 1 and 2
    input_unit = "mm",  # The distance unit in the csv file.
    efficiency = 0.90,  # The individual panel detection efficiency 
    )

The **efficiency** is defined as the **probability** for a **hit** to be **recorded** by a detector panel.

Based on the efficiency value, each hit receives either 1 (hit detected) or 0 (no detection) as a `hits_eff` variable.

In [ ]:
from muograph.plotting.plotting import plot_hist
plot_hist(hits_90eff.hits_eff.ravel(), xlabel="Hits efficiency", n_bins=10)

n_hits = (hits_90eff.n_mu * hits_90eff.n_panels)
n_detected_hits = hits_90eff.hits_eff.sum().detach().cpu().item()
n_rejected_hits =  n_hits - n_detected_hits

print(f"# detected hits = {n_detected_hits}")
print(f"# rejected hits = {n_rejected_hits}")
print(f"effective efficiency = {(n_detected_hits / n_hits) * 100 :.2f}")


The effect of efficiency on the reconstructed tracks will be treated in tutorial 3: `03_Tracking_muon_scattering_tomography.ipynb`.